In [1]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 6.6 MB/s eta 0:00:00


In [2]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [5]:
import pandas as pd
df= pd.read_csv('/content/customer_support_tickets.csv')

In [6]:
df = df.drop_duplicates()

In [7]:
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered")


In [8]:
df.columns

Index(['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age',
       'Customer Gender', 'Product Purchased', 'Date of Purchase',
       'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating'],
      dtype='object')

In [9]:
df_columns =['Ticket ID', 'Customer Name', 'Customer Email', 'Date of Purchase', 'Ticket Status',
       'Resolution', 'Ticket Priority',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating']

In [10]:
df = df.drop(columns = df_columns)

In [11]:
# Replace placeholder with actual product name
df['Ticket Description'] = df.apply(
    lambda row: row['Ticket Description'].replace("{product_purchased}", row['Product Purchased']),
    axis=1
)

In [12]:
df['Ticket Description'].head(5)

,Ticket Description
0,I'm having an issue with the GoPro Hero. Pleas...
1,I'm having an issue with the LG Smart TV. Plea...
2,I'm facing a problem with my Dell XPS. The Del...
3,I'm having an issue with the Microsoft Office....
4,I'm having an issue with the Autodesk AutoCAD....


In [13]:
df = df.drop(columns = 'Product Purchased')

In [ ]:
df.head()

,Customer Age,Customer Gender,Ticket Type,Ticket Subject,Ticket Description,Ticket Channel
0,32,Other,Technical issue,Product setup,I'm having an issue with the GoPro Hero. Pleas...,Social media
1,42,Female,Technical issue,Peripheral compatibility,I'm having an issue with the LG Smart TV. Plea...,Chat
2,48,Other,Technical issue,Network problem,I'm facing a problem with my Dell XPS. The Del...,Social media
3,27,Female,Billing inquiry,Account access,I'm having an issue with the Microsoft Office....,Social media
4,67,Female,Billing inquiry,Data loss,I'm having an issue with the Autodesk AutoCAD....,Email


In [14]:
print(df['Ticket Subject'].value_counts())

Ticket Subject
Refund request              576
Software bug                574
Product compatibility       567
Delivery problem            561
Hardware issue              547
Battery life                542
Network problem             539
Installation support        530
Product setup               529
Payment issue               526
Product recommendation      517
Account access              509
Peripheral compatibility    496
Data loss                   491
Cancellation request        487
Display issue               478
Name: count, dtype: int64


In [15]:
def fit_bertopic(texts, model_name="all-MiniLM-L6-v2", n_topics="auto"):
    embedding_model = SentenceTransformer(model_name)
    topic_model = BERTopic(embedding_model=embedding_model, nr_topics=30)
    topics, probs = topic_model.fit_transform(texts)
    return topic_model, topics, probs


In [16]:
description_model, description_topics, description_probs = fit_bertopic(df['Ticket Description'])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
#Add the topic IDs as a new column to your DataFrame
df['Topic_ID'] = description_topics

In [18]:
document_info = description_model.get_document_info(df['Ticket Description'])
df['Topic_Name'] = document_info['Name']

In [19]:
df.head()

,Customer Age,Customer Gender,Ticket Type,Ticket Subject,Ticket Description,Ticket Channel,Topic_ID,Topic_Name
0,32,Other,Technical issue,Product setup,I'm having an issue with the GoPro Hero. Pleas...,Social media,11,11_gopro_hero_camera_action
1,42,Female,Technical issue,Peripheral compatibility,I'm having an issue with the LG Smart TV. Plea...,Chat,10,10_lg_smart_oled_tv
2,48,Other,Technical issue,Network problem,I'm facing a problem with my Dell XPS. The Del...,Social media,12,12_dell_xps_pavilion_hp
3,27,Female,Billing inquiry,Account access,I'm having an issue with the Microsoft Office....,Social media,13,13_microsoft_surface_office_the
4,67,Female,Billing inquiry,Data loss,I'm having an issue with the Autodesk AutoCAD....,Email,-1,-1_to_the_my_it


In [20]:
import re

# Remove the number prefix and underscore, then replace underscores with spaces
df['Clean_Topic_Name'] = df['Topic_Name'].apply(lambda x: re.sub(r'^\d+_', '', x).replace('_', ' '))


In [21]:
df['Clean_Topic_Name'].head()

,Clean_Topic_Name
0,gopro hero camera action
1,lg smart oled tv
2,dell xps pavilion hp
3,microsoft surface office the
4,-1 to the my it


In [22]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Make sure you have the NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab') # Download the missing resource

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # 1. Lowercase
    text = text.lower()

    # 2. Remove non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)

    # 3. Tokenize
    tokens = nltk.word_tokenize(text)

    # 4. Remove stopwords and short words (optional)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]

    # 5. Lemmatize
    tokens = [lemmatizer.lemmatize(t) for t in tokens]

    # 6. Join back into string
    return ' '.join(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [23]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
df['Processed_Topic_Name'] = df['Clean_Topic_Name'].apply(preprocess_text)

In [25]:
df['Processed_Topic_Name'].head()

,Processed_Topic_Name
0,gopro hero camera action
1,smart oled
2,dell xps pavilion
3,microsoft surface office
4,


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Apply TF-IDF on the processed topic names
tfidf = TfidfVectorizer(max_features=500)  # adjust number of features if needed
X_tfidf = tfidf.fit_transform(df['Processed_Topic_Name'])


In [27]:
# Get feature names (words)
tfidf_features = tfidf.get_feature_names_out()

# Convert to DataFrame
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_features)

# Keep index same as original DataFrame
tfidf_df.index = df.index


In [28]:
df_combined = pd.concat([df, tfidf_df], axis=1)

In [29]:
df_combined.head()

,Customer Age,Customer Gender,Ticket Type,Ticket Subject,Ticket Description,Ticket Channel,Topic_ID,Topic_Name,Clean_Topic_Name,Processed_Topic_Name,...,surface,switch,thermostat,thinkpad,troubleshoot,vacuum,washing,xbox,xperia,xps
0,32,Other,Technical issue,Product setup,I'm having an issue with the GoPro Hero. Pleas...,Social media,11,11_gopro_hero_camera_action,gopro hero camera action,gopro hero camera action,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
1,42,Female,Technical issue,Peripheral compatibility,I'm having an issue with the LG Smart TV. Plea...,Chat,10,10_lg_smart_oled_tv,lg smart oled tv,smart oled,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
2,48,Other,Technical issue,Network problem,I'm facing a problem with my Dell XPS. The Del...,Social media,12,12_dell_xps_pavilion_hp,dell xps pavilion hp,dell xps pavilion,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.57735
3,27,Female,Billing inquiry,Account access,I'm having an issue with the Microsoft Office....,Social media,13,13_microsoft_surface_office_the,microsoft surface office the,microsoft surface office,...,0.604486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
4,67,Female,Billing inquiry,Data loss,I'm having an issue with the Autodesk AutoCAD....,Email,-1,-1_to_the_my_it,-1 to the my it,,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


In [30]:
df_combined.columns

Index(['Customer Age', 'Customer Gender', 'Ticket Type', 'Ticket Subject',
       'Ticket Description', 'Ticket Channel', 'Topic_ID', 'Topic_Name',
       'Clean_Topic_Name', 'Processed_Topic_Name', 'access', 'account',
       'action', 'adobe', 'airpods', 'amazon', 'apple', 'asus', 'autocad',
       'autodesk', 'bose', 'bug', 'camera', 'canon', 'charge', 'connecting',
       'controller', 'crash', 'data', 'deleted', 'dell', 'dslr', 'dyson',
       'echo', 'eos', 'file', 'fitbit', 'fix', 'flickering', 'forerunner',
       'garmin', 'google', 'gopro', 'hardware', 'hdr', 'hero', 'hue', 'iphone',
       'issue', 'kindle', 'lenovo', 'light', 'machine', 'mean', 'message',
       'microsoft', 'nest', 'network', 'nikon', 'nintendo', 'office', 'oled',
       'password', 'pavilion', 'philip', 'photoshop', 'pixel', 'playstation',
       'please', 'pro', 'robot', 'rog', 'roomba', 'samsung', 'screen', 'seems',
       'smart', 'software', 'sony', 'soundlink', 'speaker', 'surface',
       'switch', 

In [31]:
cols = ['Ticket Description','Topic_Name', 'Clean_Topic_Name', 'Processed_Topic_Name']
df_combined = df_combined.drop(columns=cols)

In [32]:
df.drop(columns='Ticket Description')

,Customer Age,Customer Gender,Ticket Type,Ticket Subject,Ticket Channel,Topic_ID,Topic_Name,Clean_Topic_Name,Processed_Topic_Name
0,32,Other,Technical issue,Product setup,Social media,11,11_gopro_hero_camera_action,gopro hero camera action,gopro hero camera action
1,42,Female,Technical issue,Peripheral compatibility,Chat,10,10_lg_smart_oled_tv,lg smart oled tv,smart oled
2,48,Other,Technical issue,Network problem,Social media,12,12_dell_xps_pavilion_hp,dell xps pavilion hp,dell xps pavilion
3,27,Female,Billing inquiry,Account access,Social media,13,13_microsoft_surface_office_the,microsoft surface office the,microsoft surface office
4,67,Female,Billing inquiry,Data loss,Email,-1,-1_to_the_my_it,-1 to the my it,
...,...,...,...,...,...,...,...,...,...
8464,22,Female,Product inquiry,Installation support,Phone,10,10_lg_smart_oled_tv,lg smart oled tv,smart oled
8465,27,Female,Technical issue,Refund request,Email,0,0_bose_samsung_soundlink_speaker,bose samsung soundlink speaker,bose samsung soundlink speaker
8466,57,Female,Technical issue,Account access,Social media,11,11_gopro_hero_camera_action,gopro hero camera action,gopro hero camera action
8467,54,Male,Product inquiry,Payment issue,Email,5,5_playstation_sony_xperia_the,playstation sony xperia the,playstation sony xperia


In [33]:
y = df_combined['Ticket Type']

In [34]:
x = df_combined.drop(columns='Ticket Type')

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test,tfidf_train,tfidf_test = train_test_split(x, y,tfidf_df ,test_size=0.3, random_state=4002)

In [36]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
categorical_features = [ 'Ticket Channel','Ticket Subject','Customer Gender']
numeric_features = ['Customer Age', 'Topic_ID']
# Create transformers for each type
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numeric_transformer = StandardScaler()
# Combine them using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
    ]
)
# Fit the preprocessor on training data only
preprocessor.fit(x_train)
x_train_processed = preprocessor.transform(x_train)
x_test_processed = preprocessor.transform(x_test)


In [37]:
# Get feature names
encoded_cat_cols = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
processed_feature_names = list(encoded_cat_cols) + numeric_features
x_train_final = pd.DataFrame(x_train_processed.toarray() if hasattr(x_train_processed, "toarray") else x_train_processed,
                             columns=processed_feature_names, index=x_train.index)
x_test_final = pd.DataFrame(x_test_processed.toarray() if hasattr(x_test_processed, "toarray") else x_test_processed,
                            columns=processed_feature_names, index=x_test.index)

In [38]:
x_train_final=pd.concat([x_train_final,tfidf_train],axis=1)
x_test_final=pd.concat([x_test_final,tfidf_test],axis=1)

In [39]:
x_test_final.head()

,Ticket Channel_Chat,Ticket Channel_Email,Ticket Channel_Phone,Ticket Channel_Social media,Ticket Subject_Account access,Ticket Subject_Battery life,Ticket Subject_Cancellation request,Ticket Subject_Data loss,Ticket Subject_Delivery problem,Ticket Subject_Display issue,...,surface,switch,thermostat,thinkpad,troubleshoot,vacuum,washing,xbox,xperia,xps
8073,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4012,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.658841,0.0,0.0
3249,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.408248,0.0,0.0,0.000000,0.0,0.0
6273,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.5,0.0,0.000000,0.0,0.0
2815,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


In [40]:
# Import libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 200],
    'max_depth': [20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [4, 6, 10]
}
# Create the grid search object
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,                     # 5-fold cross-validation
    scoring='accuracy',       # could also try 'f1', 'precision', or 'recall'
    n_jobs=-1,                # use all cores
    verbose=1                 # show progress
)
# Fit the grid search on your training data
grid_search.fit(x_train_final, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [20, None],
                         'min_samples_leaf': [4, 6, 10],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 200]},
             scoring='accuracy', verbose=1)

In [41]:
# Get the best model
best_rf_model = grid_search.best_estimator_
y_train_pred = best_rf_model.predict(x_train_final)
print(classification_report(y_train, y_train_pred))

                      precision    recall  f1-score   support

     Billing inquiry       0.65      0.63      0.64      1164
Cancellation request       0.63      0.64      0.64      1177
     Product inquiry       0.65      0.58      0.61      1154
      Refund request       0.60      0.63      0.62      1201
     Technical issue       0.62      0.67      0.64      1232

            accuracy                           0.63      5928
           macro avg       0.63      0.63      0.63      5928
        weighted avg       0.63      0.63      0.63      5928



In [42]:
y_test_pred = best_rf_model.predict(x_test_final)
print(classification_report(y_test, y_test_pred))

                      precision    recall  f1-score   support

     Billing inquiry       0.21      0.22      0.22       470
Cancellation request       0.23      0.22      0.23       518
     Product inquiry       0.18      0.16      0.17       487
      Refund request       0.22      0.21      0.21       551
     Technical issue       0.24      0.28      0.26       515

            accuracy                           0.22      2541
           macro avg       0.22      0.22      0.22      2541
        weighted avg       0.22      0.22      0.22      2541



In [53]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_final = le.fit_transform(y_train)   # fit on training labels
y_test_final = le.transform(y_test)         # just transform test labels using same mapping


In [54]:
# Import required libraries
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',   # Multi-class classification
    num_class=5,                 # Number of classes
    eval_metric='mlogloss',      # Evaluation metric
    learning_rate=0.1,
    max_depth=6,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False
)

# Fit the model on training data
xgb_model.fit(x_train_final, y_train_final)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None, num_class=5, ...)

In [55]:
# Predict on training and testing data
y_pred_train = xgb_model.predict(x_train_final)
y_pred_test = xgb_model.predict(x_test_final)

In [56]:
print("\nClassification Report:")
print(classification_report(y_train_final, y_pred_train))


Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.72      0.73      1164
           1       0.74      0.72      0.73      1177
           2       0.74      0.71      0.72      1154
           3       0.73      0.72      0.72      1201
           4       0.69      0.75      0.72      1232

    accuracy                           0.73      5928
   macro avg       0.73      0.73      0.73      5928
weighted avg       0.73      0.73      0.73      5928



In [57]:
print("\nClassification Report:")
print(classification_report(y_test_final, y_pred_test))


Classification Report:
              precision    recall  f1-score   support

           0       0.19      0.20      0.20       470
           1       0.20      0.19      0.19       518
           2       0.21      0.21      0.21       487
           3       0.21      0.20      0.21       551
           4       0.23      0.25      0.24       515

    accuracy                           0.21      2541
   macro avg       0.21      0.21      0.21      2541
weighted avg       0.21      0.21      0.21      2541



In [49]:
# Import libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Initialize Logistic Regression model
# multinomial = handles multi-class classification using softmax
# solver 'lbfgs' supports multinomial loss
log_reg = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000,   # increase iterations for convergence
    random_state=42
)

# Fit the model
log_reg.fit(x_train_final, y_train_final)

LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

In [50]:
# Predict on training and testing sets
y_pred_train = log_reg.predict(x_train_final)
y_pred_test = log_reg.predict(x_test_final)

In [51]:
print("\nClassification Report:")
print(classification_report(y_train_final, y_pred_train))


Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.25      0.25      1164
           1       0.25      0.20      0.22      1177
           2       0.24      0.20      0.22      1154
           3       0.24      0.25      0.24      1201
           4       0.26      0.33      0.29      1232

    accuracy                           0.25      5928
   macro avg       0.25      0.25      0.25      5928
weighted avg       0.25      0.25      0.25      5928



In [52]:
print("\nClassification Report:")
print(classification_report(y_test_final, y_pred_test))


Classification Report:
              precision    recall  f1-score   support

           0       0.19      0.20      0.20       470
           1       0.21      0.17      0.19       518
           2       0.16      0.15      0.15       487
           3       0.20      0.20      0.20       551
           4       0.20      0.25      0.22       515

    accuracy                           0.19      2541
   macro avg       0.19      0.19      0.19      2541
weighted avg       0.19      0.19      0.19      2541



In [58]:
# Show how labels were encoded
print("Class mapping:")
for i, class_label in enumerate(le.classes_):
    print(f"{class_label} --> {i}")


Class mapping:
Billing inquiry --> 0
Cancellation request --> 1
Product inquiry --> 2
Refund request --> 3
Technical issue --> 4
